<a href="https://colab.research.google.com/github/alxkzncoff/rds6_car_price_prediction_ml_dl/blob/main/car_price_ml_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# DEFINE
DATA_PATH                   = '/content/drive/My Drive/skill_factory_car_price_prediction_ml_dl'
RANDOM_SEED                 = 42

In [24]:
# Analyse
import pandas as pd
pd.set_option('max_columns', None, 'max_rows', None)

import numpy as np

# ML
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv(DATA_PATH+'/train.csv')
test = pd.read_csv(DATA_PATH+'/test.csv')

# Описание признаков

- **bodyType** - категориальный
- **brand** - категориальный
- **color** - категориальный
- **description** - текстовый
- **engineDisplacement** - числовой, представленный как текст
- **enginePower** - числовой, представленный как текст
- **fuelType** - категориальный
- **mileage** - числовой
- **modelDate** - числовой
- **model_info** - категориальный
- **name** - категориальный, желательно сократить размерность
- **numberOfDoors** - категориальный
- **price** - числовой, целевой
- **productionDate** - числовой
- **sell_id** - изображение (файл доступен по адресу, основанному на sell_id)
- **vehicleConfiguration** - не используется (комбинация других столбцов)
- **vehicleTransmission** - категориальный
- **Владельцы** - категориальный
- **Владение** - числовой, представленный как текст
- **ПТС** - категориальный
- **Привод** - категориальный
- **Руль** - категориальный

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6682 entries, 0 to 6681
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              6682 non-null   object 
 1   brand                 6682 non-null   object 
 2   color                 6682 non-null   object 
 3   description           6682 non-null   object 
 4   engineDisplacement    6682 non-null   object 
 5   enginePower           6682 non-null   object 
 6   fuelType              6682 non-null   object 
 7   mileage               6682 non-null   int64  
 8   modelDate             6682 non-null   int64  
 9   model_info            6682 non-null   object 
 10  name                  6682 non-null   object 
 11  numberOfDoors         6682 non-null   int64  
 12  price                 6682 non-null   float64
 13  productionDate        6682 non-null   int64  
 14  sell_id               6682 non-null   int64  
 15  vehicleConfiguration 

In [5]:
train.head(5)

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_info,name,numberOfDoors,price,productionDate,sell_id,vehicleConfiguration,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль
0,седан,BMW,чёрный,Авто на бодром ходу. Все работает отлично. П...,3.0 LTR,272 N12,бензин,245000,2007,5ER,530xi 3.0 AT (272 л.с.) 4WD,4,599000.0,2007,1099980990,SEDAN AUTOMATIC 3.0,автоматическая,3 или более,NaN,Оригинал,полный,Левый
1,седан,AUDI,серебристый,"Продажа от собственника, второй владелец, ПТС ...",2.8 LTR,204 N12,бензин,183000,2011,A6,2.8 CVT (204 л.с.),4,850000.0,2011,1095836906,SEDAN VARIATOR 2.8,вариатор,2 владельца,8 лет и 6 месяцев,Оригинал,передний,Левый
2,седан,MERCEDES,чёрный,Внимание! Только для клиентов AVILON Автомобил...,3.5 LTR,306 N12,бензин,122733,2009,E_KLASSE,350 3.5 AT (306 л.с.) 4WD,4,1325000.0,2013,1100195530,SEDAN AUTOMATIC 3.5,автоматическая,3 или более,NaN,Оригинал,полный,Левый
3,седан,AUDI,белый,В комплекте зимние колёса. 27.07.2020 Замена п...,2.0 LTR,180 N12,бензин,151000,2011,A6,2.0 CVT (180 л.с.),4,815000.0,2011,1099880662,SEDAN VARIATOR 2.0,вариатор,3 или более,4 года и 9 месяцев,Оригинал,передний,Левый
4,лифтбек,AUDI,белый,"Отличный авто, 2011 года выпуска, кроме передн...",1.8 LTR,160 N12,бензин,140000,2007,A5,1.8 CVT (160 л.с.),5,810000.0,2011,1083244610,LIFTBACK VARIATOR 1.8,вариатор,3 или более,2 года и 9 месяцев,Оригинал,передний,Левый


Построим *наивную* модель, чтобы понять в какую сторону двигаемся - к улучшению или к ухудшению.

Модель будет предсказывать среднюю цену авто по модели и году выпуска.


In [21]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [6]:
train_data, test_data = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [27]:
# Для итерации строк в pandas лучше всего использовать "Vectorization with Pandas series" и "Vectorization with NumPy arrays", так как
# они работают намного быстрее чем "iterrows()" и "apply()". В данном случае воспользуемся "iterrows()" так как датасет не такой большой

predicts = []
for index, row in pd.DataFrame(test_data[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(train_data.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(test_data['price'], predicts.values[:, 0]))*100:0.2f}%")

Точность наивной модели по метрике MAPE: 19.88%


По поводу итераций подробно расписано [здесь](https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas)